In [1]:
import json
from tqdm import tqdm
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import pandas as pd

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [2]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch_device)

cuda


In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name, device='cuda')

In [4]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "context": {"type": "text"},
            "company_id": {"type": "keyword"},
            "doc_id": {"type": "keyword"},

            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "context_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_context_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "financial-faq"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'financial-faq'})

### Encoding sentences took more than 3 hours, therefore for reusability I saved embeddings with json format into local.

In [5]:
# for doc in tqdm(documents):
#     question = doc['question']
#     context = doc['context']
#     qt = question + ' ' + context

#     doc['question_vector'] = model.encode(question)
#     doc['context_vector'] = model.encode(context)
#     doc['question_context_vector'] = model.encode(qt)

In [6]:
def convert_ndarray(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

In [7]:
# with open('documents-with-vectors.json', 'wt') as f_out:
#     json.dump(documents, f_out, indent=2, default=convert_ndarray)

In [8]:
with open('documents-with-vectors.json', 'rt') as f_in:
    documents = json.load(f_in)

def convert_to_ndarray(obj):
    if isinstance(obj, list):
        return np.array(obj)
    
documents = convert_to_ndarray(documents)
documents[0]

{'question': 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
 'context': 'NVIDIA initially focused on PC graphics.',
 'company_id': 'NVDA',
 'doc_id': '30179b8e',
 'question_vector': [0.057199183851480484,
  -0.05799635127186775,
  -0.08373435586690903,
  0.0667254850268364,
  0.06811683624982834,
  -0.07239841669797897,
  -0.07096047699451447,
  0.06869830191135406,
  -0.024992188438773155,
  0.006507430225610733,
  -0.060345035046339035,
  0.021915217861533165,
  -0.040705688297748566,
  0.02767900377511978,
  -0.043762847781181335,
  0.047260887920856476,
  -0.05507082864642143,
  -0.09983596205711365,
  -0.03174026310443878,
  -0.05299939960241318,
  -0.0014408312272280455,
  0.001925356686115265,
  0.03227229788899422,
  -0.03432941436767578,
  0.060591813176870346,
  0.050573885440826416,
  -0.05830608680844307,
  0.034611787647008896,
  0.023889508098363876,
  0.04512875899672508,
  -0.0007824070635251701,
  0.05503389984369

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 7000/7000 [01:05<00:00, 106.89it/s]


In [10]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [11]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [12]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

## Vector Search Retrieval Performance

In [13]:
def elastic_search_knn(field, vector, company_id):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "company_id": company_id
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["question", "context", "company_id", "doc_id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [14]:
def search_vector(search_function, q):
    question = q['question']
    company_id = q['company_id']

    v_q = model.encode(question)

    return elastic_search_knn(search_function, v_q, company_id)

In [15]:
def evaluate_vector(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document_id']
        results = search_vector(search_function, q)
        relevance = [d['doc_id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [19]:
evaluate_vector(ground_truth, 'question_vector')

100%|██████████| 20522/20522 [04:56<00:00, 69.19it/s]


{'hit_rate': 0.920767956339538, 'mrr': 0.8489499074164413}

In [20]:
evaluate_vector(ground_truth, 'context_vector')

100%|██████████| 20522/20522 [04:41<00:00, 73.01it/s]


{'hit_rate': 0.5987233213137121, 'mrr': 0.5064418672644008}

In [21]:
evaluate_vector(ground_truth, 'question_context_vector')

100%|██████████| 20522/20522 [04:37<00:00, 74.04it/s]


{'hit_rate': 0.9018614170158854, 'mrr': 0.8158390995029867}

## Hybrid Search Retrieval Performance

In [32]:
def elastic_search_hybrid(field, query, vector, company_id):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "company_id": company_id
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "context"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "company_id": company_id
                }
            }
        }
    }

    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "size": 5,
        "_source": ["question", "context", "company_id", "doc_id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [33]:
def search_hybrid(search_function, q):
    question = q['question']
    company_id = q['company_id']

    v_q = model.encode(question)

    return elastic_search_hybrid(search_function, question, v_q, company_id)

In [34]:
def evaluate_hybrid(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document_id']
        results = search_hybrid(search_function, q)
        relevance = [d['doc_id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [36]:
evaluate_hybrid(ground_truth, 'question_vector')

100%|██████████| 20522/20522 [05:06<00:00, 66.95it/s]


{'hit_rate': 0.9219861611928661, 'mrr': 0.9375280187116308}

In [37]:
evaluate_hybrid(ground_truth, 'context_vector')

100%|██████████| 20522/20522 [05:03<00:00, 67.67it/s]


{'hit_rate': 0.9174544391384856, 'mrr': 0.9308189585160687}

In [38]:
evaluate_hybrid(ground_truth, 'question_context_vector')

100%|██████████| 20522/20522 [05:16<00:00, 64.87it/s]


{'hit_rate': 0.9213526946691356, 'mrr': 0.9371341324757216}

````
{'hit_rate': 0.9213526946691356, 'mrr': 0.9371341324757216}
````

### Hybrid search with question_context vector performed best hit_rate and mrr results amongst other searches (vector only or hybrid with other vectors).
